In [ ]:
# default_exp utils

# Import

In [ ]:
# export
import re
import json
import time
import hashlib
from pathlib import Path

import numpy
import numba
import requests
import seaborn as sns

import ipykernel
import nbdev.export
from IPython.display import Javascript
from notebook.notebookapp import list_running_servers

In [ ]:
from IPython.core.debugger import set_trace

# Tests

`assert_allclose` checks if two things, `A` and `B`, are close to each other.

NOTE: I'm assuming the format of the inputs is the same; if not I'm assuming this is programmer error.

In [ ]:
# export
def assert_allclose(A, B, **kwargs):
    if isinstance(A, tuple):            
        for a,b in zip(A,B): assert_allclose(a, b, **kwargs) # Possibly add "strict" keyword here
    elif isinstance(A, dict):           
        for key in A.keys() | B.keys(): assert_allclose(A[key], B[key], **kwargs)
    else:
        try:    assert(numpy.allclose(A, B, **kwargs))
        except: assert(numpy.all(A == B))

In [ ]:
A = numpy.random.normal((4,3))
B = numpy.random.normal(size=(4,3))
A, B

(array([3.71269385, 4.66301704]),
 array([[ 0.27805078,  0.24093853,  0.1467166 ],
        [ 0.27100758,  3.53675506, -1.06275044],
        [-0.87741841, -2.63217781,  0.27159406],
        [ 1.12366099,  0.60327153, -0.14242279]]))

In [ ]:
assert_allclose(A, A+1e-5, atol=1e-5)
assert_allclose((A, (B, {'test': 1.})), (A+1e-5, (B+1e-5, {'test': 1 + 1e-5})), atol=1e-5)

# Image processing

In [ ]:
# export
def rgb2gray(arr): # From Pillow documentation
    return arr[:,:,0]*(299/1000) + arr[:,:,1]*(587/1000) + arr[:,:,2]*(114/1000)

# Plotting

In [ ]:
# export
def get_colors(n): return sns.color_palette(None, n)

# Notebook stuff

These are kind of hacky, but I like being able to rerun a notebook and have it auto save/build/convert at the end

In [ ]:
# export
def get_notebook_file():
    id_kernel = re.search('kernel-(.*).json', ipykernel.connect.get_connection_file()).group(1)
    for server in list_running_servers():
        response = requests.get(requests.compat.urljoin(server['url'], 'api/sessions'),
                                params={'token': server.get('token', '')})
        for r in json.loads(response.text):
            if 'kernel' in r and r['kernel']['id'] == id_kernel:
                return Path(r['notebook']['path'])

In [ ]:
assert_allclose(get_notebook_file().as_posix(), 'utils.ipynb')

In [ ]:
# export
def save_notebook():
    file_notebook = get_notebook_file()
    _get_md5 = lambda : hashlib.md5(file_notebook.read_bytes()).hexdigest() 
    md5_start = _get_md5()
    display(Javascript('IPython.notebook.save_checkpoint();')) # Asynchronous
    while md5_start == _get_md5(): time.sleep(1e-1)

In [ ]:
# export
def build_notebook(save=True):
    if save: save_notebook()
    nbdev.export.notebook2script(fname=get_notebook_file().as_posix())

In [ ]:
# export
def convert_notebook(save=True, t='markdown'):
    if save: save_notebook()
    os.system(f'jupyter nbconvert --to {t} {get_notebook_file().as_posix()}')

# Build

In [ ]:
build_notebook()

<IPython.core.display.Javascript object>

Converted utils.ipynb.
